# 4-Oscar Prediction with AutoML
After out dataframe has been assemlbed (see scraping and table_assembling) notebooks we have the data we need to make predictions on the Best Picture winner. [AutoML](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html) represents a quick, but powerful route though the Machine Learning process. H2O's AutoML runs many models through the dataset and using cross-validation, picks the best one. For my purposes I use it to confirm/compare to the Preferential Balloting Random Forest model I created.
If you are gunning to win your office's Oscar pool, scroll down to see the results.

In [2]:
import pandas as pd
import numpy as np
import h2o

# Machine Learning - Using h2o Auto ML

In [3]:
full_table = pd.read_csv('./data/processed_results/osc_df')

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.371-b11, mixed mode)
  Starting server from C:\Users\Aleksandra Czaplak\AppData\Local\Programs\Python\Python39\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ALEKSA~1\AppData\Local\Temp\tmpzura6fy_
  JVM stdout: C:\Users\ALEKSA~1\AppData\Local\Temp\tmpzura6fy_\h2o_Aleksandra_Czaplak_started_from_python.out
  JVM stderr: C:\Users\ALEKSA~1\AppData\Local\Temp\tmpzura6fy_\h2o_Aleksandra_Czaplak_started_from_python.err


c:\Users\Aleksandra Czaplak\AppData\Local\Programs\Python\Python39\lib\site-packages\h2o\backend\server.py:386: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,09 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_Aleksandra_Czaplak_jr4fw3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,241.3 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


First Year of Existance. This data will be used below
- golden_globes 1943
- pga 1989
- bafta 1960
- dga 1948
- sag 1995
- cannes 1970

In [5]:
# I pick a min_year where the awards shows will be relevant
min_year = 1995

# H2O's Auto ML

In [6]:
# Auto ML uses Cross Validation, so we do not specifiy a validation set
train = full_table.loc[((full_table['year'] < 2022) & (full_table['year'] >= min_year))]

print('training set contains:', train.shape[0], 'movies')

training set contains: 186 movies


In [7]:
train = train.drop('Unnamed: 0', axis=1)
train.columns

Index(['year', 'film', 'wiki', 'winner', 'nominations', 'Oscar_win',
       'nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy',
       'nom_pga', 'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga',
       'winner_dga', 'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes'],
      dtype='object')

In [9]:
print(type(train))

<class 'pandas.core.frame.DataFrame'>


In [8]:
from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train1 = h2o.H2OFrame(train)

# Identify predictors and response
predictors = ['year','nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy',
       'nom_pga', 'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
        'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes','nominations']

x = predictors
y = 'Oscar_win'

# For binary classification, response should be a factor
train1[y] = train1[y].asfactor()

# Run AutoML for 100 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=100, seed=1
                , keep_cross_validation_predictions= True
               , exclude_algos = ['StackedEnsemble'])

aml.train(x=x, y=y, training_frame=train1)

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
15:20:00.900: AutoML: XGBoost is not available; skipping it.
15:20:00.988: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]


15:20:02.91: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]
15:20:02.91: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 186.0.
15:20:02.98: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]

██
15:20:03.329: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]

█
15:20:03.995: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]

█
15:20:04.626: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]

██
15:20:05.417: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]

███
15:20:06.12: _tra

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
DeepLearning_grid_1_AutoML_1_20230518_152000_model_13,0.791987,0.435541,0.523313,0.232355,0.342841,0.11754,13035,0.066777,DeepLearning
DeepLearning_grid_2_AutoML_1_20230518_152000_model_9,0.745632,0.545802,0.395303,0.28232,0.360583,0.13002,11969,0.07269,DeepLearning
DeepLearning_grid_3_AutoML_1_20230518_152000_model_10,0.722339,0.513111,0.255035,0.294899,0.380841,0.14504,58032,0.099285,DeepLearning
DeepLearning_grid_1_AutoML_1_20230518_152000_model_15,0.716282,0.478913,0.426337,0.37666,0.368728,0.13596,9900,0.070182,DeepLearning
DeepLearning_grid_1_AutoML_1_20230518_152000_model_17,0.664104,0.624245,0.376196,0.382949,0.368472,0.135771,6812,0.060175,DeepLearning
DeepLearning_grid_1_AutoML_1_20230518_152000_model_9,0.658747,0.585366,0.388401,0.365129,0.388259,0.150745,10354,0.063284,DeepLearning
DeepLearning_grid_1_AutoML_1_20230518_152000_model_19,0.610762,0.595747,0.347364,0.316212,0.374992,0.140619,11594,0.06462,DeepLearning
DeepLearning_grid_2_AutoML_1_20230518_152000_model_16,0.581412,1.10088,0.243106,0.389238,0.387637,0.150263,19983,0.132074,DeepLearning
DeepLearning_grid_3_AutoML_1_20230518_152000_model_11,0.578616,0.52583,0.174457,0.37072,0.386686,0.149526,13040,0.068105,DeepLearning
DeepLearning_grid_3_AutoML_1_20230518_152000_model_8,0.564407,0.725044,0.22082,0.371069,0.374754,0.140441,46457,0.104383,DeepLearning


In [9]:
top_model = aml.leader
top_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_1_20230518_152000_model_13


Status of Neuron Layers: predicting Oscar_win, 2-class classification, bernoulli distribution, CrossEntropy loss, 852 weights/biases, 15,0 KB, 1 313 160 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ---------------------  -------------------
    1        14       Input             5.0
    2        50       RectifierDropout  40.0       0.0   0.0   0.0052818747279837095  0.027057208120822906   0.0         0.07062382860942827   0.30192017555236816  -0.2531922840566988    0.35446667671203613
    3        2        Softmax                      0.0   0.0   0.0019571759249083697  0.0036744242534041405  0.0         -0.08212572288699448  0.9244527816772461   -0.011181012423209513  0.37886369228363037

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.0606066221544762
RMSE: 0.2461841224662472
LogLoss: 0.1987397577870827
Mean Per-Class Error: 0.19461914744933612
AUC: 0.9430468204053111
AUCPR: 0.8008137032163598
Gini: 0.8860936408106221

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.35240926728521654
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      156  3    0.0189   (3.0/159.0)
1      10   17   0.3704   (10.0/27.0)
Total  166  20   0.0699   (13.0/186.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.352409     0.723404  18
max f2                       0.050816     0.766871  46
max f0point5                 0.528968     0.824176  14
max accuracy                 0.528968     0.930108  14
max precision                1            1         0
max recall                   0.0231833    1         65
max specificity              1            1         0
max absolute_mcc             0.352409     0.694518  18
max min_per_class_accuracy   0.0863697    0.851852  40
max mean_per_class_accuracy  0.050816     0.868623  46
max tns                      1            159       0
max fns                      1            25        0
max fps                      2.38047e-13  159       149
max tps                      0.0231833    27        65
max tnr                      1            1         0
max fnr                      1            0.925926  0
max fpr                      2.38047e-13  1         149
max tpr                      0.0231833    1         65

Gains/Lift Table: Avg response rate: 14,52 %, avg score: 12,01 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0107527                   1                  6.88889   6.88889            1                1            1                           1                   0.0740741       0.0740741                  588.889   588.889            0.0740741
2        0.0215054                   0.999996           6.88889   6.88889            1                1            1                           1                   0.0740741       0.148148                   588.889   588.889            0.148148
3        0.0322581                   0.99982            6.88889 

In [10]:
h2o.save_model(top_model, './basic_model')

'C:\\Users\\Aleksandra Czaplak\\Desktop\\oscars_ml\\oscar_predictions\\basic_model\\DeepLearning_grid_1_AutoML_1_20230518_152000_model_13'

In [14]:
lb.as_data_frame().to_csv('./basic_model/leaderboard.csv')

## Predict the winner

In [9]:
# Predict on 2019's films
test = full_table.loc[(full_table['year'] == 2019)]

# Import a binary outcome train/test set into H2O
test = h2o.H2OFrame(test)

# For binary classification, response should be a factor
test[y] = test[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
preds = top_model.predict(test)

preds

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict,p0,p1
0,0.872341,0.127659
0,0.871212,0.128788
0,0.871916,0.128084
0,0.871639,0.128361
0,0.872341,0.127659
0,0.872341,0.127659
1,0.508026,0.491974
0,0.871212,0.128788
0,0.805078,0.194922


In [11]:
test['pred'] = preds['predict']
test['probA'] = preds['p1']
test_pd = test.as_data_frame(use_pandas=True)

In [12]:
final_rankings = test_pd[['film','probA']].sort_values('probA', ascending = False)
final_rankings['%_confidence'] = final_rankings['probA']/final_rankings['probA'].sum() * 100
final_rankings

,film,probA,%_confidence
6,1917 (2019 film),0.491974,31.061029
8,Parasite (2019 film),0.194922,12.306475
1,The Irishman,0.128788,8.131125
7,Once Upon a Time in Hollywood,0.128788,8.131125
3,Joker (2019 film),0.128361,8.104137
2,Jojo Rabbit,0.128084,8.086673
0,Ford v Ferrari,0.127659,8.059812
4,Little Women (2019 film),0.127659,8.059812
5,Marriage Story,0.127659,8.059812


# And the Oscar goes to...

In [13]:
bp_winner = np.array(final_rankings.reset_index())[0][1].split('(')[0].strip()
print(f'And the Oscar goes to...\n🎉🏆{bp_winner}🏆🎉')

And the Oscar goes to...
🎉🏆1917🏆🎉
